# Model Calibration with Market Data

In [61]:
import pandas as pd
import scipy.optimize as scpo
import numpy as np
from rivapy.models.heston_for_DH import HestonForDeepHedging
from rivapy.models.gbm import GBM

In [62]:
data = pd.read_csv("./data/spy-options-exp-2020-08-14-weekly-show-all-stacked-08-07-2020.csv")
data = data.loc[:,['Strike', 'Bid', 'Midpoint', 'Ask',  'Type']]
data["Spread"] = (data.Ask - data.Bid)
data

,Strike,Bid,Midpoint,Ask,Type,Spread
0,160,174.31,174.48,174.65,Call,0.34
1,165,169.31,169.48,169.65,Call,0.34
2,170,164.30,164.48,164.65,Call,0.35
3,175,159.31,159.48,159.65,Call,0.34
4,180,154.30,154.48,154.65,Call,0.35
...,...,...,...,...,...,...
233,375,40.36,40.53,40.69,Put,0.33
234,380,45.37,45.53,45.68,Put,0.31
235,385,50.36,50.53,50.69,Put,0.33
236,390,55.36,55.52,55.68,Put,0.32


In [63]:
CALL = data[data.Type=="Call"]
# PUT = data[data.Type=="Put"].reset_index(drop=True)
prices = CALL.Midpoint.values
strikes = CALL.Strike.values
spreads = CALL.Spread.values
payoff = "call"

## Heston model

In [64]:
def report_calibration(initial_guess, calibrated_params):
    report = pd.DataFrame({"Initial guess": initial_guess, "Calibrated": calibrated_params},
                          index=["rho", "sigma", "theta", "kappa", "v0"]).round(4).T
    return report

In [65]:
S0=3.3433
K=S0
T=5/252
r=0.0


#r = Risk free constant rate
#rho = Correlation between stock noise and variance noise
#theta = Long term mean of the variance process
#kappa = Mean reversion coefficient for the variance process
#sigma = Volatility coefficient of the variance process
    
# Objective function
def f_Hest(x, rho, sigma, theta, kappa, v0, r=0.05):
    Hest = HestonForDeepHedging(rate_of_mean_reversion=kappa,long_run_average=theta,vol_of_vol=sigma, correlation_rho=rho,v0=v0)
    price = Hest.compute_call_price(s0=335.33, v0=Hest.v0,K=335.33,ttm=5/252)
    return price

init_vals = [-0.6, 1.0, 0.04, 2.5, 0.04] # rho, sigma, theta, kappa, v0
bounds = ( [-1, 1e-15, 1e-15, 1e-15, 1e-15], [1, np.inf, 2, np.inf, 2] )
params_Hest = scpo.curve_fit(f_Hest, strikes, prices, 
                             p0=init_vals, bounds=bounds, sigma=spreads, 
                             xtol=1e-4, max_nfev=1000)[0]

# Result
report_calibration(init_vals, params_Hest)

,rho,sigma,theta,kappa,v0
Initial guess,-0.6000,1.0000,0.0400,2.5000,0.0400
Calibrated,-0.5146,22.0943,0.0151,2.2776,0.0144


In [66]:
def Feller(x):
    return 2*x[3] * x[2] - x[1]**2 - 1e-6
cons = ({"fun": Feller, "type": "ineq"})

def least_sq(x, prices, strikes, spread):
    """ Objective function """
    Hest = HestonForDeepHedging(rate_of_mean_reversion=x[3],long_run_average=x[2],vol_of_vol=x[1], correlation_rho=x[0],v0=x[4])
    prices_calib = Hest.compute_call_price(s0=335.33, v0=Hest.v0,K=strikes,ttm=5/252)
    return np.sum( ((prices_calib - prices)/spread)**2 ) 

init_vals = [-0.4, 1.1, 0.1, 0.6, 0.02] # rho, sigma, theta, kappa, v0
bounds = ( (-1,1), (1e-15,np.inf), (1e-15, 50), (1e-15, 50), (1e-15, 10) )
params_Hest_con = scpo.minimize(least_sq, x0=init_vals, args=(prices, strikes, spreads),
                  method='SLSQP', bounds=bounds,
                  constraints=cons, tol=1e-5, options={"maxiter":1000}).x

# Result
report_calibration(init_vals, params_Hest_con)

c:\Users\Anwender\.conda\envs\rivapyFS\Lib\site-packages\scipy\optimize\_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
C:\Users\Anwender\Documents\GitHub\RiVaPy_development\RiVaPy\rivapy\models\heston_for_DH.py:118: RuntimeWarning: divide by zero encountered in scalar divide
  g = (self.rate_of_mean_reversion - self.correlation_rho*self.vol_of_vol*ixi - d) / (self.rate_of_mean_reversion - ixi * self.correlation_rho * self.vol_of_vol + d)
C:\Users\Anwender\Documents\GitHub\RiVaPy_development\RiVaPy\rivapy\models\heston_for_DH.py:118: RuntimeWarning: invalid value encountered in scalar divide
  g = (self.rate_of_mean_reversion - self.correlation_rho*self.vol_of_vol*ixi - d) / (self.rate_of_mean_reversion - ixi * self.correlation_rho * self.vol_of_vol + d)
C:\Users\Anwender\Documents\GitHub\RiVaPy_development\RiVaPy\rivapy\models\heston_for_DH.py:121: RuntimeWarning: i

,rho,sigma,theta,kappa,v0
Initial guess,-0.4000,1.1000,0.1000,0.6000,0.02
Calibrated,-0.2348,1.5573,2.6091,0.8209,0.00


In [67]:
# check the Feller condition
# 2*kappa*theta > sigma^2 is True
2*0.8209*2.6091 > 1.5573**2 >0

True

## Implied Vol, Black Scholes

In [68]:
def implied_vol_minimize(price, S0, K, T, r, payoff="call", disp=True):
    """Returns Implied volatility by minimization"""

    n = 2  # must be even

    def obj_fun(vol):
        BS = GBM(0,vol)
        prices_calib = BS.compute_call_price(S0=S0, v0=BS.v0,K=K,ttm=T)
        return (prices_calib - price) ** n

    res = scpo.minimize_scalar(obj_fun, bounds=(1e-15, 8), method="bounded")
    if res.success == True:
        return res.x
    if disp == True:
        print("Strike", K)
    return -1


In [69]:
IV_BS = []
for i in range(len(strikes)):
    IV_BS.append(implied_vol_minimize(prices[i], S0=335.33, K=strikes[i], T=5/252, r=0.))

## Check prices

In [70]:
for i in range(len(IV_BS)):
    BS = GBM(0,IV_BS[i])
    prices_BS = BS.compute_call_price(S0=335.33, v0=BS.v0,K=strikes[i],ttm=T)
    Hest = HestonForDeepHedging(rate_of_mean_reversion=0.8911,long_run_average=2.2583,vol_of_vol=1.512, correlation_rho=-0.2348,v0=0.0013)
    prices_Hest = Hest.compute_call_price(s0=335.33, v0=Hest.v0,K=strikes[i],ttm=5/252)     
    print(strikes[i],prices_BS,prices_Hest)



160 175.32999999999996 175.3300003938665
165 170.32999999999998 170.32999937306448
170 165.32999999999998 165.3300008918905
175 160.32999999999998 160.3299996871687
180 155.32999999999998 155.3299991265357
185 150.32999999999998 150.32999987495373
190 145.32999999999996 145.33000055418444
195 140.32999999999998 140.33000082468246
200 135.32999999999998 135.33000087454934
205 130.32999999999998 130.33000077615281
210 125.32999999999996 125.33000039084885
215 120.32999999999998 120.32999962189226
220 115.32999999999998 115.32999900346215
225 110.32999999999998 110.32999965314205
230 105.32999999999998 105.33000096347604
235 100.32999999999996 100.33000010483109
240 95.32999999999998 95.32999902558691
245 90.32999999999996 90.33000091557389
250 85.32999999999998 85.32999960992785
255 80.32999999999998 80.32999989134902
260 75.32999999999998 75.33000041043672
265 70.32999999999998 70.32999949855846
270 65.32999999999998 65.3300004882075
274 61.329999999999984 61.330001285376085
275 60.3299